<a href="https://colab.research.google.com/github/tohpedo/wikibot/blob/main/transformers_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SET UP


In [1]:
#colab pro stuff, checking gpu and high ram
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


  gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!
Fri Nov 19 11:36:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
    

TRAIN

In [2]:
#install depedencies hugging face
!pip install pytorch-pretrained-bert pytorch-nlp pytorch_transformers
!pip install folium

#import required packages
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig, BertModel
from pytorch_transformers import AdamW, BertForQuestionAnswering
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [3]:
#import helper functions for squad dataset
!wget 'https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad.py'
!wget 'https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad_evaluate.py'

from utils_squad import (read_squad_examples, convert_examples_to_features,
                         RawResult, write_predictions,
                         RawResultExtended, write_predictions_extended)
from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad, plot_pr_curve

--2021-11-19 11:37:01--  https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41529 (41K) [text/plain]
Saving to: ‘utils_squad.py.4’

utils_squad.py.4    100%[===================>]  40.56K  --.-KB/s    in 0.002s  

2021-11-19 11:37:02 (16.5 MB/s) - ‘utils_squad.py.4’ saved [41529/41529]

--2021-11-19 11:37:02--  https://raw.githubusercontent.com/nlpyang/pytorch-transformers/master/examples/utils_squad_evaluate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response.

In [4]:
def isGPUAvailable():
  #check if GPU is available 
  gpu = tf.test.gpu_device_name()
  if gpu != '/device:GPU:0':
    print('No GPU available')
  else:
    print('GPU is available')

isGPUAvailable()

GPU is available


In [5]:
#mount colab drive
from google.colab import drive
drive.mount('/content/drive/')
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
drive			   utils_squad_evaluate.py.2  utils_squad.py.2
__pycache__		   utils_squad_evaluate.py.3  utils_squad.py.3
sample_data		   utils_squad_evaluate.py.4  utils_squad.py.4
utils_squad_evaluate.py    utils_squad.py
utils_squad_evaluate.py.1  utils_squad.py.1


In [6]:
#set random seeds
np.random.seed(1234)
torch.manual_seed(1234)

In [7]:
#read in training data
train_data_file = '/content/drive/My Drive/train-v2.0.json'
train_data_raw = read_squad_examples(input_file=train_data_file,
                                is_training=True,
                                version_2_with_negative=True)

In [8]:
#inspecting first records
sample_records = train_data_raw[:5]
sample_records

[qas_id: 56be85543aeaaa14008c9063, question_text: When did Beyonce start becoming popular?, doc_tokens: [Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".], start_position: 39, end_position: 42,
 qas_id: 56be85543aeaaa14008c9065, question_text: What areas did Beyonce compete in when she was growing up?, doc_tokens: [Beyoncé Giselle Knowles-Carter (/biːˈ

In [9]:
#take first 75,000 rows for training due to RAM limitation
train_data_raw = train_data_raw[:30000]
train_data = pd.DataFrame.from_records([vars(record) for record in train_data_raw])
train_data.head()
print("Number of records for training: " + str(len(train_data)))

Number of records for training: 30000


In [10]:
#loooking at all questions from one article
sample = train_data.sample(frac=1).head(1)
print(sample)
context = sample.doc_tokens.values
print(context)
train_data[train_data.doc_tokens.values==context]


                         qas_id  ... is_impossible
13125  5acd431b07355d001abf3afd  ...          True

[1 rows x 7 columns]
[list(['Starting', 'at', 'the', 'transition', 'between', 'the', 'Middle', 'to', 'Upper', 'Paleolithic', 'period,', 'some', '80,000', 'to', '70,000', 'years', 'ago,', 'some', 'hunter-gatherers', 'bands', 'began', 'to', 'specialize,', 'concentrating', 'on', 'hunting', 'a', 'smaller', 'selection', 'of', '(often', 'larger)', 'game', 'and', 'gathering', 'a', 'smaller', 'selection', 'of', 'food.', 'This', 'specialization', 'of', 'work', 'also', 'involved', 'creating', 'specialized', 'tools,', 'like', 'fishing', 'nets', 'and', 'hooks', 'and', 'bone', 'harpoons.', 'The', 'transition', 'into', 'the', 'subsequent', 'Neolithic', 'period', 'is', 'chiefly', 'defined', 'by', 'the', 'unprecedented', 'development', 'of', 'nascent', 'agricultural', 'practices.', 'Agriculture', 'originated', 'and', 'spread', 'in', 'several', 'different', 'areas', 'including', 'the', 'Middle', 'East

,qas_id,question_text,doc_tokens,orig_answer_text,start_position,end_position,is_impossible
13118,56df609296943c1400a5d475,During what period did hunter-gatherers begin ...,"[Starting, at, the, transition, between, the, ...",Middle to Upper Paleolithic,6,9,False
13119,56df609296943c1400a5d476,How many years ago did hunting-gatherers start...,"[Starting, at, the, transition, between, the, ...","80,000 to 70,000 years ago",12,16,False
13120,56df609296943c1400a5d478,What was the change in gathering?,"[Starting, at, the, transition, between, the, ...",a smaller selection,26,28,False
13121,56df609296943c1400a5d479,When did the use of agriculture begin it's spr...,"[Starting, at, the, transition, between, the, ...","12,000 years ago",96,98,False
13122,5acd431b07355d001abf3afa,The transition between the Lower and Middle Pa...,"[Starting, at, the, transition, between, the, ...",,-1,-1,True
13123,5acd431b07355d001abf3afb,In what period did all hunter-gatherers begin ...,"[Starting, at, the, transition, between, the, ...",,-1,-1,True
13124,5acd431b07355d001abf3afc,The transition into the Paleolithic period is ...,"[Starting, at, the, transition, between, the, ...",,-1,-1,True
13125,5acd431b07355d001abf3afd,Agriculture ended how many years ago?,"[Starting, at, the, transition, between, the, ...",,-1,-1,True
13126,5acd431b07355d001abf3afe,"When hunter-gatherers began to specialize, the...","[Starting, at, the, transition, between, the, ...",,-1,-1,True


In [11]:
train_data['article_length'] = train_data['doc_tokens'].apply(len)
train_data['question_length'] = train_data['question_text'].apply(len)

In [12]:
max_seq_length = 256
print("Percentage of context's less than max_seq_length = %s%%" % (len([l for l in train_data['article_length'] if l <= max_seq_length])/len(train_data) * 100))

Percentage of context's less than max_seq_length = 98.82%


In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [14]:
doc_stride = 128
max_seq_length = 256
max_query_length = 64
# batch size of 64 if RAM available.
batch_size = 4


features = convert_examples_to_features(examples=train_data_raw,
                                        tokenizer=tokenizer,
                                        max_seq_length=max_seq_length,
                                        doc_stride=doc_stride,
                                        max_query_length=max_query_length,
                                        is_training=True)
torch.save(features, "/content/drive/My Drive/bert_train")


In [15]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                        all_start_positions, all_end_positions,
                        all_cls_index, all_p_mask)

In [16]:
train_sampler = RandomSampler(dataset)
train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size, drop_last=True)

In [17]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [18]:
import glob
checkpoints = sorted(glob.glob('/content/drive/My Drive/checkpoint*-[0-9]*'))

In [19]:

global_step = 0
train_loss_set = []
tr_loss = 0.0
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased')

model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

OPTIMIZER


In [20]:
param_optimizer = list(model.named_parameters())
print(param_optimizer[-2])
print(param_optimizer[-1])

('qa_outputs.weight', Parameter containing:
tensor([[ 0.0226, -0.0064, -0.0313,  ...,  0.0171, -0.0134,  0.0291],
        [ 0.0091, -0.0109, -0.0386,  ...,  0.0166, -0.0459, -0.0111]],
       device='cuda:0', requires_grad=True))
('qa_outputs.bias', Parameter containing:
tensor([0., 0.], device='cuda:0', requires_grad=True))


In [21]:
#create AdamW optimizer for fine tuning
learning_rate = 5e-5
#learning_rate = 0.01
adam_epsilon=1e-8
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)

In [ ]:
num_train_epochs = 1

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

print("***** Running training *****")
print("  Num examples = %d" % len(dataset))
print("  Num Epochs = %d" % num_train_epochs)
print("  Batch size = %d" % batch_size)
print("  Total optimization steps = %d" % (len(train_dataloader) // num_train_epochs))

model.zero_grad()
train_iterator = trange(num_train_epochs, desc="Epoch")

for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
      if step < global_step + 1:
        continue

      model.train()
      batch = tuple(t.to(device) for t in batch)

      inputs = {'input_ids':       batch[0],
                'attention_mask':  batch[1], 
                'token_type_ids':  batch[2],  
                'start_positions': batch[3], 
                'end_positions':   batch[4]}

      outputs = model(**inputs)

      loss = outputs[0]
      train_loss_set.append(loss)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      tr_loss += loss.item()
      optimizer.step()
      model.zero_grad()
      global_step += 1
    
      if global_step % 1000 == 0:
        print("Train loss: {}".format(tr_loss/global_step))
        output_dir = '/content/drive/My Drive/checkpoint-{}'.format(global_step)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        torch.save(torch.tensor(train_loss_set), os.path.join(output_dir, 'training_loss.pt'))
        print("Saving model checkpoint to %s" % output_dir)

***** Running training *****
  Num examples = 32302
  Num Epochs = 1
  Batch size = 4
  Total optimization steps = 8075


Iteration:   0%|          | 0/8075 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)

Iteration:  12%|█▏        | 956/8075 [07:18<54:27,  2.18it/s]

In [ ]:
output_dir = '/content/drive/My Drive/checkpoint-final'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)

In [ ]:
train_loss_set_ckpt = torch.load('/content/drive/My Drive/checkpoint-final/training_loss.pt')
train_loss_set = to_list(train_loss_set_ckpt)

In [ ]:
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

In [ ]:
test_data = '/content/drive/My Drive/dev-v2.0.json'
val_examples = read_squad_examples(input_file=test_data,
                                is_training=False,
                                version_2_with_negative=True
                                )

doc_stride = 128
max_seq_length = 256
max_query_length = 64
cached_features_file = '/content/drive/My Drive/cache_validation'


# Cache features for faster loading
if not os.path.exists(cached_features_file):
  features = convert_examples_to_features(examples=val_examples,
                                        tokenizer=tokenizer,
                                        max_seq_length=max_seq_length,
                                        doc_stride=doc_stride,
                                        max_query_length=max_query_length,
                                        is_training=False)
  torch.save(features, cached_features_file)
else:
  features = torch.load(cached_features_file)

In [ ]:
# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)

all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                        all_example_index, all_cls_index, all_p_mask)

In [ ]:
validation_sampler = SequentialSampler(dataset)
validation_dataloader = DataLoader(dataset, sampler=validation_sampler, batch_size=batch_size, drop_last=True)

TEST MODEL

In [ ]:
def evaluate(model, tokenizer):
  print("***** Running evaluation *****")
  print("  Num examples = %d" % len(dataset))
  print("  Batch size = %d" % batch_size)
  all_results = []
  predict_file = '/content/drive/My Drive/dev-v2.0.json'
  for batch in tqdm(validation_dataloader, desc="Evaluating", miniters=100, mininterval=5.0):
    model.eval()
    batch = tuple(t.to(device) for t in batch)
    with torch.no_grad():
      inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
      example_indices = batch[3]
      outputs = model(**inputs)

    for i, example_index in enumerate(example_indices):
      eval_feature = features[example_index.item()]
      unique_id = int(eval_feature.unique_id)

      result = RawResult(unique_id    = unique_id,
                         start_logits = to_list(outputs[0][i]),
                         end_logits   = to_list(outputs[1][i]))
      all_results.append(result)

  # Compute predictions
  output_prediction_file = "/content/drive/My Drive/predictions.json"
  output_nbest_file = "/content/drive/My Drive/nbest_predictions.json"
  output_null_log_odds_file = "/content/drive/My Drive/null_odds.json"
  output_dir = "/content/drive/My Drive/predict_results"

  write_predictions(val_examples, features, all_results, 10,
                  30, True, output_prediction_file,
                  output_nbest_file, output_null_log_odds_file, False,
                  True, 0.0)

  # Evaluate with the official SQuAD script
  evaluate_options = EVAL_OPTS(data_file=predict_file,
                               pred_file=output_prediction_file,
                               na_prob_file=output_null_log_odds_file,
                               out_image_dir=None)
  results = evaluate_on_squad(evaluate_options)
  return results

In [ ]:
results = evaluate(model, tokenizer)

In [ ]:
import sklearn

param_grid = dict(epochs=[1,10,25])
grid = sklearn.model_selection.GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(train_data)




